In [7]:
import pandas as pd
import numpy as np
import xlrd
import plotly.express as px

# Mapping of PSOC Code to ISCO Code

In [8]:
# Get the data
filename = 'data/2022-Updates-to-the-2012-PSOC.xlsx'
relevant_cols = [2, 5]
names = ['PSOC', 'ISCO']
df_maps = pd.read_excel(
    filename, 
    usecols=relevant_cols, 
    names=names,
    sheet_name=None,
    dtype={"PSOC": str, "ISCO": str}
    )

# for each df_map in df_maps, get their jobs to ISCO pairs
psoc_isco = {}
for _, df_map in df_maps.items():
    df_map.dropna(inplace=True)
    mapping = dict(zip(df_map['PSOC'], df_map['ISCO']))
    psoc_isco.update(mapping)

# Using FINAL (WIP)-MCA Job list to ISCO

There are two sheets of interest for us for the Excel file *FINAL (WIP)-MCA Job list August 2025.xlsx*. Namely, they are **USE_final** and **Pass 3 - Sector reports**. The former is important because it contains the final list of jobs that should have the orderer pair of the exposure v complementarity scores. The latter is an auxilliary sheet that contains the PSOC codes of jobs. What we noticed is that the intersection of the jobs between these two are over 97%.

In [9]:
# Opening the MCA Job list
filename = 'data/FINAL (WIP)-MCA Job list August 2025.xlsx'

# make a dataframe for the final job list
final_jobs_df = pd.read_excel(filename, sheet_name=0)
final_jobs = set(final_jobs_df['Job Title'].to_list())

# make another dataframe for the sheet with PSOC codes
pass_df = pd.read_excel(filename, sheet_name=4)
pass_jobs = set(pass_df['Job Title'].to_list())

# Show the Jaccard Index
inter = len(final_jobs.intersection(pass_jobs))
union = len(final_jobs.union(pass_jobs))
print(f'The two lists share over {round(inter/union, 2) * 100}% of jobs in common.')

The two lists share over 98.0% of jobs in common.


In [10]:
# Create the mapping from the job to PSOC
job_psoc = dict(zip(pass_df['Job Title'].str.strip(), pass_df['PSOC Code']))
final_jobs_df['PSOC'] = final_jobs_df['Job Title'].apply(
    lambda x : job_psoc.get(x, np.nan)
    )
print("Only 15 jobs do not have a PSOC Code. "
      "My simple answer would be to just drop, "
      "but I don't know if that is the best solution.")

final_jobs_df.isnull().sum()

Only 15 jobs do not have a PSOC Code. My simple answer would be to just drop, but I don't know if that is the best solution.


Job Title                                                  0
Job Sector                                                 0
Job Subsector                                              0
Educational Pathway                                        0
HEI with PRC (Professional Regulation Commission) Exam     0
Some HEI                                                   0
PSOC                                                      15
dtype: int64

In [11]:
clean_jobs_df = final_jobs_df.dropna()
clean_jobs_df = clean_jobs_df.copy()
clean_jobs_df['ISCO'] = clean_jobs_df['PSOC'].apply(lambda x: psoc_isco.get(str(int(x)), np.nan))
clean_jobs_df[clean_jobs_df['ISCO'].isnull()][['Job Title', 'PSOC', 'ISCO']]

,Job Title,PSOC,ISCO
74,Automotive Mechanic Specialized In EV Power,7414.0,NaN
162,Chemical Sprayer,6110.0,NaN
246,Co-op Administrative Support,4111.0,NaN
344,Embassy Administrative Attaché,4111.0,NaN
659,Livestock Agriculture Technician,3144.0,NaN
741,Ministry Program Assistant,4111.0,NaN
983,Seminary Support Staff,4111.0,NaN


# ISCO to O*NET

In [12]:
# Get the mapping from ISCO codes to SOC codes
filename = 'data/isco_soc_crosswalk.xls'
isco_soc_df = pd.read_excel(filename,
                            usecols=[0, 3],
                            names=['ISCO', 'SOC'],
                            skiprows=6,
                            dtype={'ISCO': str, 'SOC': str})
isco_soc = dict(zip(isco_soc_df['ISCO'], isco_soc_df['SOC']))

clean_jobs_df['SOC'] = clean_jobs_df['ISCO'].apply(lambda x : isco_soc.get(x, np.nan))

In the process of mapping the ISCO codes to SOC codes, 25 jobs were lost. In total from this whole process of the MCA job list to the SOC codes, 47 jobs were dropped. For an overall visualization of how the jobs were dropped, please see the funnel chart below.

In [13]:
import plotly.express as px

stages = [
    'Original MCA Job List', 
    'Jobs to PSOC Codes', 
    'PSOC Codes to ISCO Codes', 
    'ISCO Codes to O*NET-SOC Codes'
]
numbers = [1151, 1136, 1129, 1104]
data = dict(number=numbers, stage=stages)

# Show number of jobs + percentage on the bar
percent_text = [f"{stage}<br>{n/numbers[0]*100:.2f}%" 
                for stage, n in zip(stages, numbers)]

fig = px.funnel(
    data, 
    x='number', 
    y='stage', 
    text=percent_text
)

# Set bar color
fig.update_traces(marker_color="#be2328")  # dark red color

# Set background to white and font to Times New Roman
fig.update_layout(
    paper_bgcolor='white',
    plot_bgcolor='white',
    font=dict(
        family="Times New Roman, Times, serif",
        size=14,
        color="black"
    )
)

# Hide y-axis entirely
fig.update_yaxes(showticklabels=False, title='')

fig.show()


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




To deal with jobs that do not have a SOC code, we asked ChatGPT to find the most suitable SOC code. We have attached below a shareable link of the conversation for transparency.
https://chatgpt.com/share/68ca0dd0-46e0-8012-be0a-e825db6ec12f

In [16]:
missing_soc = {
    "Abaca Grower": "45-2092",  # Farmworkers and Laborers, Crop, Nursery, and Greenhouse
    "Ambulance Driver": "53-3011",  # Ambulance Drivers and Attendants, Except Emergency Medical Technicians
    "Aquaculture Business Assistant": "11-9013",  # Farmers, Ranchers, and Other Agricultural Managers
    "Company Driver (Hotel)": "53-3053",  # Shuttle Drivers and Chauffeurs
    "Delivery Driver": "43-5021",  # Couriers and Messengers
    "Fishery Operations Assistant": "45-2093",  # Farmworkers, Farm, Ranch, and Aquacultural Animals
    "Fitness Instructor": "39-9031",  # Exercise Trainers and Group Fitness Instructors
    "Halal Seaweed Processing Worker": "51-9199",  # Production Workers, All Other
    "Hatchery Farm Worker": "45-2093",  # Farmworkers, Farm, Ranch, and Aquacultural Animals
    "Livestock Raiser": "45-2093",  # Farmworkers, Farm, Ranch, and Aquacultural Animals
    "Municipal Fisherman": "45-3031",  # Fishing and Hunting Workers
    "Park and Garden Rangers": "33-9099",  # Protective Service Workers, All Other
    "Poultry Farmer": "45-2093",  # Farmworkers, Farm, Ranch, and Aquacultural Animals
    "Professional Driver": "53-3053",  # Shuttle Drivers and Chauffeurs
    "Seaweed Nursery Operator": "45-2092",  # Farmworkers and Laborers, Crop, Nursery, and Greenhouse
    "Seaweed Processor": "51-9199",  # Production Workers, All Other
    "Service Vehicle Drivers": "53-3053",  # Shuttle Drivers and Chauffeurs
    "Sonar Fishing Operator": "45-3031",  # Fishing and Hunting Workers
    "Sports Coach": "27-2022",  # Coaches and Scouts
    "Sports Program Assistant": "39-9031",  # Exercise Trainers and Group Fitness Instructors
    "Swine Raiser/Farmer": "45-2093",  # Farmworkers, Farm, Ranch, and Aquacultural Animals
    "Taxi Drivers": "53-3054",  # Taxi Drivers and Chauffeurs
    "Tilapia Farm Technician": "45-2093",  # Farmworkers, Farm, Ranch, and Aquacultural Animals
    "TVET Instructor": "25-1194",  # Vocational Education Teachers, Postsecondary
    "Wellness Coach": "39-9031",  # Exercise Trainers and Group Fitness Instructors
    "Automotive Mechanic Specialized In EV Power": "49-3023",  # Automotive Service Technicians and Mechanics
    "Chemical Sprayer": "37-3012.00",  # Pesticide Handlers, Sprayers, and Applicators, Vegetation
    "Co-op Administrative Support": "43-601",  # Secretaries and Administrative Assistants, Except Legal, Medical, and Executive
    "Embassy Administrative Attaché": "43-6011",  # Executive Secretaries and Executive Administrative Assistants
    "Livestock Agriculture Technician": "19-4012",  # Agricultural Technicians
    "Ministry Program Assistant": "43-406",  # Eligibility Interviewers, Government Programs
    "Seminary Support Staff": "21-2099"  # Religious Workers, All Other
}

In [17]:
for title, code in list(missing_soc.items()):
    job_filt = clean_jobs_df['Job Title'] == title
    clean_jobs_df.loc[job_filt, 'SOC'] = code

In [18]:
clean_jobs_df.isnull().sum()

Job Title                                                 0
Job Sector                                                0
Job Subsector                                             0
Educational Pathway                                       0
HEI with PRC (Professional Regulation Commission) Exam    0
Some HEI                                                  0
PSOC                                                      0
ISCO                                                      7
SOC                                                       0
dtype: int64

# Since we now have all the jobs mapped to their corresponding SOC code, let us now export it.

In [19]:
clean_jobs_df.to_csv('mca_soc.csv', index=False)